In [2]:
import pandas as pd

df = pd.read_csv("courses_croissance_revenu.csv", encoding ="utf-8")

Le but de cette analyse est de d'identifier les facteurs qui rendent les incitations (remise de bienvenue, points de fidélité, prime objectif) plus rentables pour une société fictive de transport à la demande. Connaître les facteurs qui augmentent le revenu permettront un meilleur ciblage des "receiveurs d'incitations". Les facteurs examinés sont la situation géographique(zone de départ), les differents segments de clients et de chauffeurs, la période (heure de pointe, weekend ou semaine de travail)...

L'analyse répondra à des questions exploratoires puis déterminera la combinaison de facteurs la plus rentable par incitation. Une conclusion ou une recommendation sera donnée après chaque analyse.

Q1 — Quelles incitations génèrent le plus de revenu net pour l’entreprise ?

Parmi les différentes incitations proposées (remise_bienvenue, prime_objectif, points_gowin), lesquelles génèrent le revenu net le plus élevé pour l'entreprise ?

In [3]:
#Filtrage des courses terminées
df_terminees = df[df["statut_course"] == "terminee"].copy()

df_terminees


,id_course,id_chauffeur,id_client,date_heure_demande,jour_semaine,heure,est_weekend,est_pointe,depart,arrivee,...,note_chauffeur,segment_chauffeur,incitation,points_gagnes,points_utilises_reduction,cout_incitation,recette_course,revenu_net,revenu_moyen_segment,hausse_revenu_positive
0,C00001,CH0058,CL0403,2025-02-03 23:00:00,lun,23,0,0,Zopah,Menontin,...,4.80,performant,prime_objectif,0,0,300.00,659.60,359.60,800.0,0
1,C00002,CH0013,CL0632,2025-03-27 19:32:00,jeu,19,0,1,Zopah,Akpakpa,...,4.47,intermediaire,points_gowin,33,17,17.00,676.32,659.32,800.0,0
2,C00003,CH0141,CL0758,2025-03-08 03:06:00,sam,3,1,0,Gbodjè,Haie Vive,...,4.66,intermediaire,remise_bienvenue,0,0,317.60,1701.94,1384.34,1200.0,1
3,C00004,CH0126,CL0307,2025-02-24 03:06:00,lun,3,0,0,Haie Vive,Haie Vive,...,3.97,intermediaire,aucune,0,0,0.00,560.27,560.27,1200.0,0
4,C00005,CH0115,CL0608,2025-01-15 07:00:00,mer,7,0,1,Agla,Haie Vive,...,4.68,intermediaire,points_gowin,72,34,34.00,1566.85,1532.85,1200.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,C01996,CH0264,CL0747,2025-03-01 09:01:00,sam,9,1,1,Gbodjè,Gbodjè,...,4.20,intermediaire,remise_bienvenue,0,0,292.08,1476.01,1183.93,1200.0,0
1996,C01997,CH0072,CL0344,2025-03-28 08:17:00,ven,8,0,1,Gbodjè,Menontin,...,4.15,intermediaire,points_gowin,45,21,21.00,1001.40,980.40,1200.0,0
1997,C01998,CH0168,CL0458,2025-01-07 10:56:00,mar,10,0,0,Gbodjè,Zopah,...,4.11,performant,aucune,0,0,0.00,838.62,838.62,1200.0,0
1998,C01999,CH0164,CL0012,2025-01-06 09:14:00,lun,9,0,1,Haie Vive,Menontin,...,4.11,intermediaire,aucune,0,0,0.00,322.95,322.95,2000.0,0


In [4]:
#Calcul des indicateurs

q1_summary =(

df_terminees.groupby("incitation").agg(
    nb_courses = ("id_course", "count"),
    revenu_moyen = ("revenu_net", "mean"),
    revenu_median = ("revenu_net", "median"),
    revenu_total =("revenu_net", "sum"),
    cout_incitation_moyen =("cout_incitation", "mean")
)
.sort_values("revenu_moyen", ascending=False)

)

print(q1_summary)

                  nb_courses  revenu_moyen  revenu_median  revenu_total  \
incitation                                                                
points_gowin             337   1389.234214       1150.520     468171.93   
aucune                   834   1286.152542       1141.115    1072651.22   
remise_bienvenue         345   1068.715884        872.550     368706.98   
prime_objectif           274   1023.291861        756.935     280381.97   

                  cout_incitation_moyen  
incitation                               
points_gowin                  24.240356  
aucune                         0.000000  
remise_bienvenue             257.093739  
prime_objectif               300.000000  


Les points gozem génèrent le plus de revenu pour un coût d'investissement assez faible (24 f) contrairement aux autres incitation. C'est l'incitation la plus rentable et celle sur qui se basera la suite de nos analyses. Vu que les deux autres incitation sont moins rentables, elles doivent êtres attribuées plus prudemment pour ne pas réduire le revenu.

Q2- Les points Gowin sont-ils plus efficaces sur certains segments clients ?

In [5]:
#Filtrage des courses terminées et avec l'incentive points gowin

df_points = df_terminees[df_terminees["incitation"] == "points_gowin"].copy()

df_points

,id_course,id_chauffeur,id_client,date_heure_demande,jour_semaine,heure,est_weekend,est_pointe,depart,arrivee,...,note_chauffeur,segment_chauffeur,incitation,points_gagnes,points_utilises_reduction,cout_incitation,recette_course,revenu_net,revenu_moyen_segment,hausse_revenu_positive
1,C00002,CH0013,CL0632,2025-03-27 19:32:00,jeu,19,0,1,Zopah,Akpakpa,...,4.47,intermediaire,points_gowin,33,17,17.0,676.32,659.32,800.0,0
4,C00005,CH0115,CL0608,2025-01-15 07:00:00,mer,7,0,1,Agla,Haie Vive,...,4.68,intermediaire,points_gowin,72,34,34.0,1566.85,1532.85,1200.0,1
16,C00017,CH0014,CL0340,2025-01-28 15:09:00,mar,15,0,0,Jéricho,Haie Vive,...,4.31,faible_activite,points_gowin,32,0,0.0,678.23,678.23,1200.0,0
18,C00019,CH0102,CL0113,2025-02-09 03:00:00,dim,3,1,0,Agla,Akpakpa,...,5.00,intermediaire,points_gowin,24,12,12.0,488.16,476.16,800.0,0
31,C00032,CH0173,CL0593,2025-01-16 14:19:00,jeu,14,0,0,Fidjrossè,Cadjehoun,...,4.93,intermediaire,points_gowin,49,33,33.0,949.06,916.06,1200.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1967,C01968,CH0038,CL0719,2025-02-02 20:25:00,dim,20,1,0,Menontin,Gbodjè,...,4.14,intermediaire,points_gowin,84,43,43.0,1699.22,1656.22,800.0,1
1986,C01987,CH0021,CL0244,2025-03-07 18:20:00,ven,18,0,1,Akpakpa,Menontin,...,4.49,intermediaire,points_gowin,24,12,12.0,543.99,531.99,800.0,0
1988,C01989,CH0020,CL0328,2025-02-09 13:35:00,dim,13,1,0,Menontin,Gbodjè,...,4.67,intermediaire,points_gowin,27,4,4.0,584.98,580.98,800.0,0
1991,C01992,CH0243,CL0091,2025-01-15 13:14:00,mer,13,0,0,Gbodjè,Fidjrossè,...,4.36,intermediaire,points_gowin,57,2,2.0,1166.03,1164.03,800.0,1


In [6]:
# Calcul des indicateurs

q2_summary =(
    df_points.groupby("segment_client").agg
    (revenu_moyen = ("revenu_net","mean"),
     revenu_median = ("revenu_net","median"),
     revenu_total = ("revenu_net", "sum")
     ).
     sort_values("revenu_moyen", ascending=False)

)

print(q2_summary)

                revenu_moyen  revenu_median  revenu_total
segment_client                                           
inactif          1465.375205       1189.965     213944.78
nouveau_client   1340.477548       1091.910     207774.02
regulier         1290.364722       1205.555      46453.13


Le résultat de cette analyse montrent que les clients inactifs génèrent plus de revenus avec l'utilisation de points gozem. C'est une irrégularité de cette base de données synthétique. Je vais continuer mon analyse en excluant les clients inactif. Ce qui nécessite un nouveau filtrage de df_points.

In [7]:
# Refiltrage de df_points

df_points_actifs = df_points[df_points["segment_client"].isin(["nouveau_client", "regulier"])].copy()

df_points_actifs

,id_course,id_chauffeur,id_client,date_heure_demande,jour_semaine,heure,est_weekend,est_pointe,depart,arrivee,...,note_chauffeur,segment_chauffeur,incitation,points_gagnes,points_utilises_reduction,cout_incitation,recette_course,revenu_net,revenu_moyen_segment,hausse_revenu_positive
4,C00005,CH0115,CL0608,2025-01-15 07:00:00,mer,7,0,1,Agla,Haie Vive,...,4.68,intermediaire,points_gowin,72,34,34.0,1566.85,1532.85,1200.0,1
16,C00017,CH0014,CL0340,2025-01-28 15:09:00,mar,15,0,0,Jéricho,Haie Vive,...,4.31,faible_activite,points_gowin,32,0,0.0,678.23,678.23,1200.0,0
31,C00032,CH0173,CL0593,2025-01-16 14:19:00,jeu,14,0,0,Fidjrossè,Cadjehoun,...,4.93,intermediaire,points_gowin,49,33,33.0,949.06,916.06,1200.0,0
46,C00047,CH0151,CL0698,2025-01-29 07:17:00,mer,7,0,1,Calavi Kpota,Calavi Kpota,...,4.12,intermediaire,points_gowin,153,104,104.0,3002.61,2898.61,1200.0,1
69,C00070,CH0126,CL0723,2025-03-31 01:40:00,lun,1,0,0,Calavi Tokpa,Fidjrossè,...,4.58,intermediaire,points_gowin,133,27,27.0,2641.71,2614.71,1200.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1907,C01908,CH0055,CL0225,2025-02-03 08:45:00,lun,8,0,1,Calavi Kpota,Menontin,...,4.33,faible_activite,points_gowin,80,30,30.0,1711.49,1681.49,1200.0,1
1919,C01920,CH0197,CL0332,2025-02-07 09:15:00,ven,9,0,1,Jéricho,Calavi Kpota,...,4.51,intermediaire,points_gowin,79,7,7.0,1540.94,1533.94,1200.0,1
1930,C01931,CH0240,CL0075,2025-02-22 10:45:00,sam,10,1,0,Akpakpa,Haie Vive,...,4.32,intermediaire,points_gowin,75,15,15.0,1567.83,1552.83,1200.0,1
1959,C01960,CH0206,CL0315,2025-03-12 23:00:00,mer,23,0,0,Agla,Menontin,...,4.36,intermediaire,points_gowin,41,14,14.0,786.98,772.98,1200.0,0


In [8]:
# Calcul des indicateurs

q3_summary =(
   
   df_points_actifs.groupby("segment_client").agg(
       nb_courses = ("id_course", "count"),
       revenu_moyen = ("revenu_net", "mean"),
       revenu_median = ("revenu_net", "median"),
       revenu_total  = ("revenu_net", "sum")
   ).sort_values("revenu_moyen", ascending = False)
)

print(q3_summary)

                nb_courses  revenu_moyen  revenu_median  revenu_total
segment_client                                                       
nouveau_client         155   1340.477548       1091.910     207774.02
regulier                36   1290.364722       1205.555      46453.13


A première vue, on pourrait dire que les nouveaux clients rapportent plus que les clients réguliers mais on ne peut pas vraiment tirer cette conclusion car le nombre de trajets par les nouveaux clients est beaucoup plus élevé dans cet échantillon de données. Je vais donc faire une analyse avec l'indicateur suivant 

Q3- Les points Gowin sont-ils plus rentables avec certains moyens de paiement ?

In [9]:
#Calcul des indicateurs

q4_summary = (
    df_points_actifs.groupby("moyen_paiement").agg(
        nb_trajets = ("id_course","count"),
        revenu_moyen = ("revenu_net", "mean"),
        revenu_median = ("revenu_net", "median"),
        revenu_total = ("revenu_net", "sum")
    ).sort_values("revenu_moyen", ascending=False)
)

print(q4_summary)

                nb_trajets  revenu_moyen  revenu_median  revenu_total
moyen_paiement                                                       
cash                    62   1494.166613        1178.71      92638.33
portefeuille            77   1314.588571        1196.26     101223.32
carte                   52   1160.875000         962.67      60365.50


L'analyse a déterminé que le portefeuille est le moyen de paiement le plus rentable combiné à l'incitation points gowin. En se basant non pas sur le revenu moyen mais sur le revenu median qui est plus representatif, j'ai pu déterminer que le portefeuille était plus rentable et plus fiable.

La médiane décrit ce qui se passe la plupart du temps, alors que la moyenne peut être dominée par quelques cas exceptionnels.

Q4- Quelles sont les zones de départ les plus rentables lorsque les points Gowin sont utilisés ?

Après avoir identifié l’incitation (points Gowin) et le moyen de paiement stratégique, la zone de départ est le levier marketplace le plus important : c’est là que l’offre et la demande se rencontrent et que le prix se forme.

Dans un souci d'optimisation, je refiltre df_points_actifs pour ne garder que les courses avec portefeuille comme moyen de paiement. La table que j'obtiendrai comportera des courses effectuées par des utilisateurs actifs, ayant utilisé des points gozem et ayant utilisé leur portefeuille électronique comme moyen de paiement.


In [10]:
#Filtrage 

df_zone = df_points_actifs[
    df_points_actifs["moyen_paiement"] == "portefeuille"
].copy()

df_zone


,id_course,id_chauffeur,id_client,date_heure_demande,jour_semaine,heure,est_weekend,est_pointe,depart,arrivee,...,note_chauffeur,segment_chauffeur,incitation,points_gagnes,points_utilises_reduction,cout_incitation,recette_course,revenu_net,revenu_moyen_segment,hausse_revenu_positive
16,C00017,CH0014,CL0340,2025-01-28 15:09:00,mar,15,0,0,Jéricho,Haie Vive,...,4.31,faible_activite,points_gowin,32,0,0.0,678.23,678.23,1200.0,0
31,C00032,CH0173,CL0593,2025-01-16 14:19:00,jeu,14,0,0,Fidjrossè,Cadjehoun,...,4.93,intermediaire,points_gowin,49,33,33.0,949.06,916.06,1200.0,0
142,C00143,CH0277,CL0279,2025-02-15 00:03:00,sam,0,1,0,Calavi Tokpa,Haie Vive,...,4.67,intermediaire,points_gowin,40,25,25.0,851.26,826.26,1200.0,0
144,C00145,CH0001,CL0112,2025-01-26 21:15:00,dim,21,1,0,Calavi Kpota,Fidjrossè,...,4.26,intermediaire,points_gowin,121,32,32.0,2567.81,2535.81,1200.0,1
166,C00167,CH0271,CL0006,2025-01-30 02:53:00,jeu,2,0,0,Cadjehoun,Menontin,...,4.70,intermediaire,points_gowin,109,36,36.0,2112.49,2076.49,1200.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,C01796,CH0178,CL0052,2025-01-08 21:09:00,mer,21,0,0,Zopah,Jéricho,...,4.34,intermediaire,points_gowin,84,12,12.0,1743.51,1731.51,1200.0,1
1817,C01818,CH0283,CL0632,2025-03-11 11:31:00,mar,11,0,0,Fidjrossè,Gbodjè,...,4.77,intermediaire,points_gowin,43,9,9.0,839.52,830.52,1200.0,0
1874,C01875,CH0296,CL0783,2025-03-08 04:51:00,sam,4,1,0,Jéricho,Zopah,...,4.70,faible_activite,points_gowin,42,29,29.0,863.60,834.60,1200.0,0
1907,C01908,CH0055,CL0225,2025-02-03 08:45:00,lun,8,0,1,Calavi Kpota,Menontin,...,4.33,faible_activite,points_gowin,80,30,30.0,1711.49,1681.49,1200.0,1


In [11]:
#Calcul des indicateurs

zone_summary = (
    df_zone
    .groupby("depart")
    .agg(
        nb_courses=("id_course", "count"),
        revenu_median=("revenu_net", "median"),
        revenu_moyen=("revenu_net", "mean"),
        revenu_total=("revenu_net", "sum")
    )
    .sort_values("revenu_median", ascending=False)
)

zone_summary


,nb_courses,revenu_median,revenu_moyen,revenu_total
depart,,,,
Gbodjè,8,1575.260,1926.327500,15410.62
Calavi Kpota,8,1568.970,1577.475000,12619.80
Agla,3,1483.790,1742.810000,5228.43
Cadjehoun,7,1355.860,1210.015714,8470.11
Akpakpa,7,1215.300,1285.535714,8998.75
Fidjrossè,7,1196.260,1408.020000,9856.14
Sainte-Rita,4,967.690,962.042500,3848.17
Haie Vive,6,880.700,1305.280000,7831.68
Zopah,4,830.030,942.835000,3771.34


Les 5 zones les plus rentables (avec l'utilisation de points gowin et le paiement par portefeuille) sont

Gbodjè, Calavi Kpota, Cadjehoun, Akpakpa et Fidjrossè

L’analyse met en évidence que certaines zones de départ, notamment Gbodjè et Calavi Kpota, présentent un revenu net médian significativement plus élevé lorsque les points Gowin sont combinés au paiement par portefeuille. Ces zones génèrent une valeur typique élevée par course, indiquant que l’incitation n’érode pas la rentabilité. À l’inverse, des zones comme Menontin, Jéricho ou Calavi Tokpa affichent une rentabilité typique plus faible et ne constituent pas des cibles prioritaires pour une stratégie d’optimisation du revenu.

Q5 - L'incitation points gozem est elle vraiment plus rentable en weekend et aux heures de pointe?

In [17]:
q6_summary = (
    df_zone
    .groupby("est_weekend")
    .agg(
        nb_courses=("id_course", "count"),
        revenu_median=("revenu_net", "median"),
        revenu_moyen=("revenu_net", "mean"),
        revenu_total=("revenu_net", "sum")
    )
).sort_values("revenu_median", ascending= False)

q6_summary


,nb_courses,revenu_median,revenu_moyen,revenu_total
est_weekend,,,,
0,51,1196.26,1245.104902,63500.35
1,26,1090.86,1450.883462,37722.97


In [18]:
q7_summary = (
    df_zone
    .groupby("est_pointe")
    .agg(
        nb_courses=("id_course", "count"),
        revenu_median=("revenu_net", "median"),
        revenu_moyen=("revenu_net", "mean"),
        revenu_total=("revenu_net", "sum")
    )
).sort_values("revenu_median", ascending= False)

q7_summary


,nb_courses,revenu_median,revenu_moyen,revenu_total
est_pointe,,,,
1,14,1414.805,1484.75500,20786.57
0,63,1005.740,1276.77381,80436.75


D'après mes analyses, les transports sont beaucoup plus rentables pendant le weekend et les heures de pointe

Q7- Les points Gowin sont-ils plus efficaces avec certains segments de chauffeurs ?

In [19]:
#Calcul des indicateurs

q7_chauffeur = (
    df_points_actifs
    .groupby("segment_chauffeur")
    .agg(
        nb_courses=("id_course", "count"),
        revenu_median=("revenu_net", "median"),
        revenu_moyen=("revenu_net", "mean"),
        revenu_total=("revenu_net", "sum")
    )
    .sort_values("revenu_median", ascending=False)
)

q7_chauffeur


,nb_courses,revenu_median,revenu_moyen,revenu_total
segment_chauffeur,,,,
faible_activite,19,1623.650,1593.508947,30276.67
performant,26,1330.015,1566.698077,40734.15
intermediaire,146,1008.260,1254.906370,183216.33


Bien que les chauffeurs intermédiaires contribuent majoritairement au revenu total en raison de leur volume, les chauffeurs performants génèrent un revenu net significativement plus élevé par course. Les incitations devraient donc être prioritairement ciblées vers les chauffeurs performants afin de maximiser le retour sur investissement.